In [1]:
import pandas as pd
import uuid
from utils.game import fetch_monthly_games

In [2]:
start_year = 1996
end_year = 2024
blacklist = [(2020, 'april'), (2021, 'october'), (2021, 'november')]
months = ['january', 'february', 'march', 'april', 'may', 'june', 'october', 'november', 'december']

df = pd.DataFrame(columns=['Date', 'season_start_year', 'Visitor/Neutral', 'PTS', 'Home/Neutral', 'PTS.1', 'Unnamed: 5', 'Unnamed: 6', 'Attend.', 'Notes'])

df = fetch_monthly_games(df, start_year, end_year, months, blacklist)


Processing games for year, month: (1996, january)
Processing games for year, month: (1996, february)
Processing games for year, month: (1996, march)
Processing games for year, month: (1996, april)
Processing games for year, month: (1996, may)
Processing games for year, month: (1996, june)
Processing games for year, month: (1996, october)
404 encountered for URL: https://www.basketball-reference.com/leagues/NBA_1996_games-october.html. Skipping.
Processing games for year, month: (1996, november)
Processing games for year, month: (1996, december)
Processing games for year, month: (1997, january)
Processing games for year, month: (1997, february)
Processing games for year, month: (1997, march)
Processing games for year, month: (1997, april)
Processing games for year, month: (1997, may)
Processing games for year, month: (1997, june)
Processing games for year, month: (1997, october)
404 encountered for URL: https://www.basketball-reference.com/leagues/NBA_1997_games-october.html. Skipping.


In [3]:
print(df)
secret_df = df.copy() # In case of emergencies
df.count()

                  Date season_start_year       Visitor/Neutral  PTS  \
0     Tue, Jan 2, 1996              1995   Seattle SuperSonics  111   
1     Tue, Jan 2, 1996              1995             Utah Jazz  102   
2     Tue, Jan 2, 1996              1995        Indiana Pacers  102   
3     Tue, Jan 2, 1996              1995    Philadelphia 76ers   90   
4     Tue, Jan 2, 1996              1995       Houston Rockets  105   
..                 ...               ...                   ...  ...   
217  Sat, Dec 31, 2022              2022      Dallas Mavericks  126   
218  Sat, Dec 31, 2022              2022  New Orleans Pelicans  101   
219  Sat, Dec 31, 2022              2022       Detroit Pistons  116   
220  Sat, Dec 31, 2022              2022    Philadelphia 76ers  115   
221  Sat, Dec 31, 2022              2022            Miami Heat  126   

               Home/Neutral PTS.1 Unnamed: 5 Unnamed: 6 Attend. Notes  \
0             Atlanta Hawks    88  Box Score        NaN    8524   NaN   
1

Date                 35097
season_start_year    35097
Visitor/Neutral      35097
PTS                  35097
Home/Neutral         35097
PTS.1                35097
Unnamed: 5            5838
Unnamed: 6           29603
Attend.              35083
Notes                  163
Arena                35097
Start (ET)           29259
Unnamed: 7            1797
dtype: int64

In [4]:
"""
The covid season has two october months, which the code above does not find. I add these months by iterating through this url_lst.
"""
url_lst = ['https://www.basketball-reference.com/leagues/NBA_2020_games-october-2019.html',
          'https://www.basketball-reference.com/leagues/NBA_2020_games-october-2020.html']
for url in url_lst:
    yearMonth_table = pd.io.html.read_html(url)[0]
    yearMonth_table['season_start_year'] = 2019
    df = pd.concat([df, yearMonth_table])

In [5]:
#Fix date column
df_backup = df.copy()
df = df_backup.copy()
df = df[df.Date!='Playoffs'].copy()
df['datetime'] = pd.to_datetime(df['Date'], infer_datetime_format=True)

C:\Users\dkste\AppData\Local\Temp\ipykernel_21636\1340948566.py:5: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['datetime'] = pd.to_datetime(df['Date'], infer_datetime_format=True)


In [7]:
"""
Creates a variable for playoff and regular season games.
"""
playOffStart = {1995:'1996-04-25', 1996:'1997-04-24', 1997:'1998-04-23', 1998:'1999-05-08', 1999:'2000-04-22', 2000:'2001-04-21',
               2001:'2002-04-20',2002:'2003-04-19', 2003:'2004-04-17', 2004:'2005-04-23', 2005:'2006-04-22',
               2006: '2007-04-21', 2007:'2008-04-19', 2008:'2009-04-18', 2009:'2010-04-17', 2010:'2011-04-16',
               2011:'2012-04-28', 2012:'2013-04-20', 2013:'2014-04-19', 2014:'2015-04-18', 2015:'2016-04-16',
               2016:'2017-04-15', 2017:'2018-04-14', 2018:'2019-04-13', 2019:'2020-08-15', 2020:'2021-05-18', 2021:'2022-04-16', 2022:'2023-04-15'}

df['playoff'] = df['season_start_year'].apply(lambda x: playOffStart[x])
df['is_regular'] = df['playoff'] > df['datetime']
df['is_regular'] = df['is_regular'].apply(lambda x: 1 if x else 0)
df.drop(columns=['playoff'], inplace=True)

In [8]:
# Remove exhibition games in 2018-19
df[(df.season_start_year==2018)&(df.is_regular==1)&(df['Visitor/Neutral']=='Boston Celtics')].sort_values(by=['datetime'])
df['d'] = df['datetime']> '2018-10-01'
df['y'] = df['season_start_year'] != 2018
df = df[(df['d'])|(df['y'])].copy()
df.drop(columns=['d','y'], inplace=True)

In [9]:
"""The 2012 season has 1 fewer games due to the bombing at the Boston Marathon. 
Otherwise all the seasons have the correct number of games."""

df[(df['is_regular']==1)]['season_start_year'].value_counts()

season_start_year
2022    1236
2021    1236
2007    1230
2018    1230
2017    1230
2016    1230
2015    1230
2014    1230
2013    1230
2010    1230
2008    1230
2009    1230
2006    1230
2005    1230
2004    1230
2012    1229
1996    1189
2003    1189
2002    1189
2001    1189
2000    1189
1999    1189
1997    1189
1995    1189
2020    1080
2011     990
2019     971
1998     725
Name: count, dtype: int64

In [10]:
# I drop Unnecessary columns
df.drop(columns=['Unnamed: 5','Unnamed: 6', 'Unnamed: 7', 'Date'], inplace=True)

In [11]:
#Create game_id
df = df.sort_values(by='datetime')
df['game_id'] = [str(uuid.uuid4()) for i in range(len(df))]

In [12]:
#Set column names
df = df.set_axis(['season_start_year', 'away_team', 'points_away', 'home_team', 'points_home','attendance',
            'notes', 'start_et', 'arena', 'datetime', 'is_regular', 'game_id'], axis=1)

df = df.drop(columns=['arena'])
df = df[df['season_start_year'] > 2018]

In [13]:
#Upload with new file name
df.to_csv('games.csv', index = False)